In [1]:
import pandas as pd
import numpy as np
import string
import random
import collections
import os
os.environ['DATASET_DIR'] = "/home/rustambaku13/Documents/Warwick/d3-gnn/jupyter/datasets/"

# Helpers

In [2]:
def get_out_degree(data):
    """ data Array of size D, 2. Where d is number of uniEdges and src, dest """
    v = {}
    for src, dest in data:
        v[src] = v.get(dest,0) + 1
    return sum(v.values()) / len(v) if len(v) > 0 else 0

def get_running_out_degree(data):
    l = []
    for i in range(1000, data.shape[0], 1000):
        if i % 10000 == 0:
            print(i)
        l.append(get_out_degree(data[[0,1]][:i].values))
    return sum(l) / len(l) if len(l) > 0 else 0

def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

# Tag-Ask-Ubuntu

In [25]:
def create_tag_ask_ubuntu(cap=70000):
    n_vertices = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-nverts.txt"), header=None)[0].values
    simplices = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-simplices.txt"), header=None)[0].values
    n_labels = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-node-labels.txt"), header=None, delimiter=" ", usecols=[1])[1].values
    simplex_labels = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-simplex-labels.txt"), header=None, delimiter=" ")[0].values
    simplex_dict = collections.defaultdict(list) # simplex -> [nodes]
    reverse_dict = collections.defaultdict(list) # nodes -> [simplex]
    index = 0
    for simplex_idx in range(n_vertices.shape[0]):
        if simplex_idx > cap:
            break
        s = str(simplex_labels[simplex_idx])
        for j in simplices[index:index+n_vertices[simplex_idx]]:
            n_label = n_labels[j-1]
            reverse_dict[n_label].append(s)
            simplex_dict[s].append(n_label)
        index+=n_vertices[simplex_idx]
    return (reverse_dict, simplex_dict, index)

def create_file_pairwise_edges(my_dict, destination):
    connections = list()
    with open(destination,"w") as f:
        for key, values in my_dict.items():
            for value in values:
                f.write(f'{key},{value}\n')
                f.write(f'{value},{key}\n')


def create_file(my_dict, destination):
    with open(destination,"w") as f:
        for key, val in my_dict.items():
            f.write(f'{key},{",".join(val)}\n')

In [33]:
a,b,c = create_tag_ask_ubuntu(1000000000)
create_file(b, os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-simplex-node.txt"))
create_file(a, os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-node-simplex.txt"))

In [24]:
create_file_pairwise_edges(a, os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-pairwise-edges.csv"))

# Reddit-hyperlink

In [5]:
res = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "RedditHyperlinks","soc-redditHyperlinks-body.tsv"), header=None, delimiter="\t", usecols=[0,1])

## Means

In [6]:
res

,0,1
0,leagueoflegends,teamredditteams
1,theredlion,soccer
2,inlandempire,bikela
3,nfl,cfb
4,playmygame,gamedev
...,...,...
286556,negareddit,debatefascism
286557,mildlynomil,justnomil
286558,mmorpg,blackdesertonline
286559,electricskateboards,askreddit


In [7]:
acc = 0unique i,j in res.iterrows():
    acc += res[:i][res[:i][0] == j[1]].shape[0]
acc = acc / res.shape[0]
acc, res.groupby(0).count()[1].mean()

(121.25355857915069, 10.28464271614686)

In [ ]:
res[0].uniq

# StackOverflow

In [5]:
data = pd.read_csv("../datasets/StackOverflow/sx-stackoverflow.txt", header=None, delimiter=" ")
data.to_csv("../datasets/StackOverflow/sx-stackoverflow.tsv", index=False, header=False)

In [4]:
dataset = dgl.data.RedditDataset()
graph = dataset[0]

In [7]:
graph.ndata["feat"].element_size() * graph.ndata["feat"].nelement()

560979720

In [ ]:

os.environ["DATASET_DIR"] = "/home/rustambaku13/Documents/Warwick/flink-streaming-gnn/jupyter/datasets"
frame = pd.DataFrame({"src":graph.uniEdges()[0], "dest":graph.uniEdges()[1]})
frame.to_csv(os.path.join(os.getenv("DATASET_DIR"), "reddit", "graph.tsv"), header=None, index=None, sep="\t")
np.save(os.path.join(os.getenv("DATASET_DIR"), "reddit", "node_features"), graph.ndata["feat"])
np.save(os.path.join(os.getenv("DATASET_DIR"), "reddit", "node_labels"), graph.ndata["label"])